In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
%%time
data = fetch_20newsgroups(subset='all', categories=['comp.graphics', 'sci.med'])

In [ ]:
data['target_names']

In [ ]:
texts = data['data']
target = data['target']

In [ ]:
print(texts[0])

In [ ]:
data['target_names'][target[0]]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(encoding='utf8', min_df=1)
vectorizer.fit(texts)

In [ ]:
vectorizer.transform(texts[:1])

In [ ]:
print(vectorizer.transform(texts[:1]).indices)
print(vectorizer.transform(texts[:1]).data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(encoding='utf8', min_df=1)
vectorizer.fit(texts)

In [ ]:
vectorizer.transform(texts[:1])

In [ ]:
print(vectorizer.transform(texts[:1]).indices)
print(vectorizer.transform(texts[:1]).data)

In [ ]:
import nltk
stemmer = nltk.stem.snowball.RussianStemmer()

In [ ]:
print(stemmer.stem('машинное'), stemmer.stem('машинный'))

In [ ]:
print(stemmer.stem('машина'), stemmer.stem('машины'))

In [ ]:
stemmer = nltk.stem.snowball.EnglishStemmer()

def stem_text(text, stemmer):
    tokens = text.split()
    return ' '.join(map(lambda w: stemmer.stem(w), tokens))

stemmed_texts = []
for t in tqdm(texts[:1000]):
    stemmed_texts.append(stem_text(t, stemmer))

In [ ]:
print(texts[0])

In [ ]:
print(stemmed_texts[0])

In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

In [ ]:
morph.parse('машинное')[0]

In [ ]:
morph.parse('машинные')[0]

In [ ]:
morph.parse('стали')

In [ ]:
morph.parse('стали специалистом')

In [ ]:
morph.parse('сплав стали')

Сравним работу стеммера и лемматизатора на примере:

In [ ]:
stemmer = nltk.stem.snowball.RussianStemmer()
print(stemmer.stem('машинное'))

In [ ]:
print(morph.parse('машинное')[0].normal_form)

In [ ]:
print(stemmer.stem('машинная'))

In [ ]:
print(morph.parse('машинная')[0].normal_form)

In [ ]:
print(stemmer.stem('машина'))

In [ ]:
print(morph.parse('машина')[0].normal_form)

In [ ]:
data = pd.read_csv('house_prices.csv')

In [ ]:
data.head()

In [ ]:
data = data.drop(columns=['Id'])
y = data['SalePrice']
X = data.drop(columns=['SalePrice'])

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.distplot(y, label='target')
plt.title('target')

plt.subplot(1, 2, 2)
sns.distplot(data.GrLivArea, label='area')
plt.title('area')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=10)

numeric_data = X_train.select_dtypes([np.number])
numeric_data_mean = numeric_data.mean()
numeric_features = numeric_data.columns

X_train = X_train.fillna(numeric_data_mean)[numeric_features]
X_test = X_test.fillna(numeric_data_mean)[numeric_features]

In [ ]:
sns.distplot(np.log(y+1), label='target')
plt.show()

In [ ]:
model = Ridge()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Test RMSE = %.4f' % mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
model = Ridge()
model.fit(X_train, np.log(y_train+1))
y_pred = np.exp(model.predict(X_test))-1

print('Test RMSE = %.4f' % mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
X_train['GrLivArea'] = np.log(X_train['GrLivArea'] + 1)
X_test['GrLivArea'] = np.log(X_test['GrLivArea'] + 1)

In [ ]:
model = Ridge()
model.fit(X_train[numeric_features], y_train)
y_pred = model.predict(X_test[numeric_features])

print('Test RMSE = %.4f' % mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
model = Ridge()
model.fit(X_train[numeric_features], np.log(y_train+1))
y_pred = np.exp(model.predict(X_test[numeric_features]))-1

print('Test RMSE = %.4f' % mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
from sklearn.linear_model import LinearRegression

np.random.seed(36)
X = np.random.uniform(0, 1, size=100)
y = np.cos(1.5 * np.pi * X) + np.random.normal(scale=0.1, size=X.shape)

In [ ]:
plt.scatter(X, y)

In [ ]:
X = X.reshape((-1, 1))
thresholds = np.arange(0.2, 1.1, 0.2).reshape((1, -1))

X_expand = np.hstack((
    X,
    ((X > thresholds[:, :-1]) & (X <= thresholds[:, 1:])).astype(int)))

In [ ]:
thresholds

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
-np.mean(cross_val_score(
    LinearRegression(), X, y, cv=KFold(n_splits=3),
    scoring='neg_mean_squared_error'))

In [ ]:
-np.mean(cross_val_score(
    LinearRegression(), X_expand, y, cv=KFold(n_splits=3),
    scoring='neg_mean_squared_error'))

In [ ]:
X

In [ ]:
X_expand